In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from observational_fear import load
from observational_fear.events import get_block_starts

from binit.bin import which_bin_idx
from binit.align import align_around
from neurobox.correlations import pairwise_correlation, correlation_matrix_to_tidy
from neurobox.wide_transforms import resample
from observational_fear.nbox_transforms import summarize_prepost_events
from observational_fear.stats import auc



DATA_DIR = Path(r"F:\OFL\one-p experiment\data")
FIG_DIR = Path(r"C:\Users\roryl\repos\observational-fear\figs")

In [14]:
def corr_by_trial(
    df, events, time_before=0, time_after=5, time_col="time",
):
    def correlate(df, time_col="aligned", neuron_col="new_id", value_col="value"):
        df_piv = df.pivot(index=time_col, columns=neuron_col, values=value_col)
        df_corr = pairwise_correlation(df_piv, rectify=False, zero_diag=True)
        df_tidy = correlation_matrix_to_tidy(df_corr)
        out = dict(mean_correlation=df_tidy["value"].abs().mean(), sem=df_tidy["value"].abs().std() / np.sqrt(len(df_tidy["value"])))
        return pd.Series(out)

    df["trial_idx"] = which_bin_idx(
        df[time_col].values,
        bin_edges=events,
        time_before=time_before,
        time_after=time_after,
    )
    df["aligned"] = align_around(
        df[time_col].values, events, t_before=time_before, max_latency=time_after
    )
    df["aligned"] = np.round(df["aligned"], 2)
    return (
        df.groupby("trial_idx")
        .apply(correlate)
        .reset_index()
    )


def run_corr_by_trial(data_dir, session, block, coreg_only):
    df_traces = load.load_traces_long(session=session, data_dir=data_dir, coreg_only=coreg_only)
    events = get_block_starts(session=session, block_name=block)
    res_pre = corr_by_trial(
        df_traces, events, time_before=5, time_after=0, time_col="time",
    ).assign(prepost="pre")
    res_post = corr_by_trial(
        df_traces, events, time_before=0, time_after=5, time_col="time",
    ).assign(prepost="post")
    return pd.concat([res_pre, res_post]).assign(block=block, session=session)


def run_prepost_by_trial(data_dir, session, block, coreg_only):
    df_traces = load.load_traces_long(session=session, data_dir=data_dir, coreg_only=coreg_only)
    events = get_block_starts(session=session, block_name=block)
    df_res = summarize_prepost_events(
        df=df_traces,
        events=events,
        t_before=5,
        t_after=5,
        summary_func=auc,
        cell_col="new_id",
    )
    df_res = pd.melt(
        df_res, id_vars=["_trial", "new_id"], var_name="prepost", value_name="auc"
    ).pivot(index="_trial", columns=["prepost", "new_id"], values="auc")
    return df_res

In [15]:
DATA_DIR

WindowsPath('F:/OFL/one-p experiment/data')

In [18]:
derived_data_dir = DATA_DIR / "derived" / "correlations"
derived_data_dir.mkdir(exist_ok=True)

exports = [
    dict(session="day2", block="CS", coreg=False),
    dict(session="day2", block="US", coreg=True),
    dict(session="day4", block="US", coreg=True),
]

for export in tqdm(exports):
    res = run_prepost_by_trial(data_dir=DATA_DIR, session=export["session"], block=export["block"], coreg_only=export["coreg"])
    res.to_csv(derived_data_dir / f"AUC PrePost By Trial - {export['session']} - {export['block']}.csv")

  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
df_list = []
for export in tqdm(exports):
    df_list.append(run_corr_by_trial(data_dir=DATA_DIR, session=export["session"], block=export["block"], coreg_only=export["coreg"]))

df = pd.concat(df_list).reset_index(drop=True)

df.to_csv(derived_data_dir / f"Correlations by Trial.csv")

  0%|          | 0/3 [00:00<?, ?it/s]